# Merge all bills files and cleanup data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import stats
import os
import json
from json.decoder import JSONDecodeError
import glob
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
import scipy.stats as st


In [2]:
#Read all output data files and add them to a data frame

path = r'output_data/' # use your path
all_files = glob.glob(path + "/bills*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

bills_df = pd.concat(li, axis=0, ignore_index=True)

In [3]:
bills_df.head()

,bill_slug,congress,bill_id,bill_type,bill_uri,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_state,...,house_passage,senate_passage,enacted,vetoed,cosponsors,cosponsors_by_party_D,cosponsors_by_party_R,committees,committee_codes,primary_subject
0,hr1,112,hr1-112,hr,https://api.propublica.org/congress/v1/112/bil...,NaN,NaN,R000395,Harold Rogers,KY,...,2011-02-19,2011-03-09,NaN,NaN,0.0,NaN,NaN,House Appropriations; House Budget,[],NaN
1,hr2,112,hr2-112,hr,https://api.propublica.org/congress/v1/112/bil...,Repealing the Job-Killing Health Care Law Act,Rep.,C001046,Eric Cantor,VA,...,2011-01-19,NaN,NaN,NaN,182.0,NaN,182.0,House Energy and Commerce; House Education and...,[],Health
2,hr3,112,hr3-112,hr,https://api.propublica.org/congress/v1/112/bil...,No Taxpayer Funding for Abortion Act,Rep.,S000522,Christopher H. Smith,NJ,...,2011-05-04,NaN,NaN,NaN,227.0,12.0,216.0,House Judiciary; House Energy and Commerce; Ho...,[],Health
3,hr4,112,hr4-112,hr,https://api.propublica.org/congress/v1/112/bil...,Comprehensive 1099 Taxpayer Protection and Rep...,Rep.,L000517,Dan Lungren,CA,...,2011-03-03,NaN,NaN,NaN,273.0,37.0,236.0,House Ways and Means,[],Taxation
4,hr5,112,hr5-112,hr,https://api.propublica.org/congress/v1/112/bil...,Protecting Access to Healthcare Act,Rep.,G000550,Phil Gingrey,GA,...,2012-03-22,NaN,NaN,NaN,134.0,3.0,133.0,House Judiciary; House Energy and Commerce,[],Health


In [4]:
#Check for dup bills in data set by bill id
 
dup_bills=bills_df.groupby(["bill_id"]).size().sort_values(ascending=False).reset_index(name="Frequency")
dup_bills=dup_bills.loc[dup_bills["Frequency"]>1]
print("-----Duplicate bills-----")
print(dup_bills)

# Create a clean DataFrame by dropping the duplicate bill by its ID.
clean_bills_df = bills_df.drop_duplicates(subset=(["bill_id"]), keep='first')
clean_bills_df.head()

-----Duplicate bills-----
Empty DataFrame
Columns: [bill_id, Frequency]
Index: []


,bill_slug,congress,bill_id,bill_type,bill_uri,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_state,...,house_passage,senate_passage,enacted,vetoed,cosponsors,cosponsors_by_party_D,cosponsors_by_party_R,committees,committee_codes,primary_subject
0,hr1,112,hr1-112,hr,https://api.propublica.org/congress/v1/112/bil...,NaN,NaN,R000395,Harold Rogers,KY,...,2011-02-19,2011-03-09,NaN,NaN,0.0,NaN,NaN,House Appropriations; House Budget,[],NaN
1,hr2,112,hr2-112,hr,https://api.propublica.org/congress/v1/112/bil...,Repealing the Job-Killing Health Care Law Act,Rep.,C001046,Eric Cantor,VA,...,2011-01-19,NaN,NaN,NaN,182.0,NaN,182.0,House Energy and Commerce; House Education and...,[],Health
2,hr3,112,hr3-112,hr,https://api.propublica.org/congress/v1/112/bil...,No Taxpayer Funding for Abortion Act,Rep.,S000522,Christopher H. Smith,NJ,...,2011-05-04,NaN,NaN,NaN,227.0,12.0,216.0,House Judiciary; House Energy and Commerce; Ho...,[],Health
3,hr4,112,hr4-112,hr,https://api.propublica.org/congress/v1/112/bil...,Comprehensive 1099 Taxpayer Protection and Rep...,Rep.,L000517,Dan Lungren,CA,...,2011-03-03,NaN,NaN,NaN,273.0,37.0,236.0,House Ways and Means,[],Taxation
4,hr5,112,hr5-112,hr,https://api.propublica.org/congress/v1/112/bil...,Protecting Access to Healthcare Act,Rep.,G000550,Phil Gingrey,GA,...,2012-03-22,NaN,NaN,NaN,134.0,3.0,133.0,House Judiciary; House Energy and Commerce,[],Health


In [5]:
#Check for invalid dataset/unuseful dataset  based on bill introduced date

invalid_data=clean_bills_df['introduced_date'].isna().groupby(clean_bills_df.congress, sort=False).sum().reset_index()
invalid_list=invalid_data.loc[invalid_data['introduced_date'] == True]
invalid_list

,congress,introduced_date
0,112,True


In [6]:
#drop bills with congress number in invalid list
clean_bills_df = clean_bills_df[~clean_bills_df['congress'].isin(invalid_list['congress'])]
clean_bills_df.head()

,bill_slug,congress,bill_id,bill_type,bill_uri,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_state,...,house_passage,senate_passage,enacted,vetoed,cosponsors,cosponsors_by_party_D,cosponsors_by_party_R,committees,committee_codes,primary_subject
6725,hr1,113,hr1-113,hr,https://api.propublica.org/congress/v1/113/bil...,Tax Reform Act of 2014,Rep.,C000071,Dave Camp,MI,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,House Ways and Means Committee,['HSWM'],Taxation
6726,hr2,113,hr2-113,hr,https://api.propublica.org/congress/v1/113/bil...,American Energy Solutions for Lower Costs and ...,Rep.,T000459,Lee Terry,NE,...,2014-09-18,NaN,NaN,NaN,15.0,NaN,15.0,House Energy and Commerce Committee,"['HSII', 'HSSY', 'HSJU', 'HSPW', 'HSIF']",Energy
6727,hr3,113,hr3-113,hr,https://api.propublica.org/congress/v1/113/bil...,Northern Route Approval Act,Rep.,T000459,Lee Terry,NE,...,2013-05-22,NaN,NaN,NaN,134.0,2.0,132.0,House Natural Resources Committee,"['HSII', 'HSPW', 'HSIF']",Energy
6728,hr4,113,hr4-113,hr,https://api.propublica.org/congress/v1/113/bil...,Jobs for America Act,Rep.,C000071,Dave Camp,MI,...,2014-09-18,NaN,NaN,NaN,4.0,NaN,4.0,House Judiciary Committee,"['HSWM', 'HSSM', 'HSII', 'HSAG', 'HSBA', 'HSJU...",Economics and Public Finance
6729,hr5,113,hr5-113,hr,https://api.propublica.org/congress/v1/113/bil...,Student Success Act,Rep.,K000363,John Kline,MN,...,2013-07-19,NaN,NaN,NaN,12.0,NaN,12.0,"Senate Health, Education, Labor, and Pensions ...","['SSHR', 'HSED', 'HSBA']",Education


In [7]:
#Write the clean data into a csv file to be used for the project
output_data_file = "output_data/clean_bills.csv"
output_path = os.path.join(output_data_file)
clean_bills_df.to_csv(output_path, index=False, header=True)